In [6]:
import numpy as np 
import cv2
import os
from os import listdir
import shutil
import itertools
import imutils
from tqdm import tqdm
from PIL import Image

In [7]:
def crop_brain_contour(image, plot=False):
    
    # Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    

    # Find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]            

    if plot:
        plt.figure()

        plt.subplot(1, 2, 1)
        plt.imshow(image)
        
        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        
        plt.title('Original Image')
            
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)

        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)

        plt.title('Cropped Image')
        
        plt.show()
    
    return new_image


In [8]:
def save_new_images(x_set, y_set, folder_name):
    os.makedirs(folder_name, exist_ok=True)
    for label in ['1', '2', '3']:
        os.makedirs(os.path.join(folder_name, label), exist_ok=True)
    
    for i, (img, imclass) in enumerate(zip(x_set, y_set)):
        cv2.imwrite(f"{folder_name}/{imclass}/{i}.png", img)

In [10]:
labels = ['1', '2', '3']  # Corresponding to 1: Meningioma, 2: Glioma, 3: Pituitary tumor

X_train = []
y_train = []
X_test = []
y_test = []

print("Train Set")
print("===========")
image_size = 240
for label in labels:
    folderPath = os.path.join(r'C:\_D\Uni\Minor Project\brain-tumor-classification\dataset\split_images', 'train', label)
    for file in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath, file))
        img = crop_brain_contour(img)
        img = cv2.resize(img, (image_size, image_size))  # Resize image to the same shape
        X_train.append(img)
        y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

print("Test Set")
print("=========")
for label in labels:
    folderPath = os.path.join(r'C:\_D\Uni\Minor Project\brain-tumor-classification\dataset\split_images', 'test', label)
    for file in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath, file))
        img = crop_brain_contour(img)
        img = cv2.resize(img, (image_size, image_size))  # Resize image to the same shape
        X_test.append(img)
        y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)

Train Set


100%|██████████| 744/744 [00:02<00:00, 370.77it/s]


Test Set


100%|██████████| 93/93 [00:00<00:00, 106.48it/s]


In [11]:
# Save cropped images
save_new_images(X_train, y_train, folder_name=r'C:\_D\Uni\Minor Project\brain-tumor-classification\dataset\cropped\train')
save_new_images(X_test, y_test, folder_name=r'C:\_D\Uni\Minor Project\brain-tumor-classification\dataset\cropped\test')

print("Cropped images saved successfully!")

Cropped images saved successfully!
